In [1]:
class Parent():
    def __init__(self, a=1, b=2, c=3):
        self.a=a
        self.b=b
        self.c=c
    def _print(self, s):
        print(f"I`m parent: a={self.a}\tb={self.b}\tc={self.c}\n{s}")
    
    def __call__(self, s):
        self._print(s)

class Child(Parent):
    def _print(self, s):
        print(f"I`m child: a={self.a}\tb={self.b}\tc={self.c}\n{s}")


In [2]:
child = Child(a=11)
child("woca")

I`m child: a=11	b=2	c=3
woca


In [1]:
import torch
import torch.nn as nn
from torchinfo import summary
class CoupledHead(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.layer = nn.Conv2d(in_c, out_c, 3, 1, 1)
    def forward(self, x):
        return self.layer(x)


class DecoupledHead(nn.Module):
    def __init__(self, in_c, out_c, split_ratio):
        super().__init__()
        out_c_1 = int(out_c * split_ratio)
        out_c_2 = out_c - out_c_1
        self.layer1 = nn.Conv2d(in_c, out_c_1, 3, 1, 1)
        self.layer2 = nn.Conv2d(in_c, out_c_2, 3, 1, 1)
    def forward(self, x):
        x1 = self.layer1(x)
        x2 = self.layer2(x)
        out = torch.cat([x1, x2], dim=1)
        return out


In [7]:
# img = torch.randn(2,64,5120, 5120)
m1 = CoupledHead(64, 64)
summary(m1, [2, 64, 2000, 2000])

Layer (type:depth-idx)                   Output Shape              Param #
CoupledHead                              --                        --
├─Conv2d: 1-1                            [2, 64, 2000, 2000]       36,928
Total params: 36,928
Trainable params: 36,928
Non-trainable params: 0
Total mult-adds (G): 295.42
Input size (MB): 2048.00
Forward/backward pass size (MB): 4096.00
Params size (MB): 0.15
Estimated Total Size (MB): 6144.15

In [8]:
m2 = DecoupledHead(64, 64, 0.3)
summary(m2, [2, 64, 1000, 1000])

RuntimeError: CUDA out of memory. Tried to allocate 490.00 MiB (GPU 0; 10.92 GiB total capacity; 5.39 GiB already allocated; 183.00 MiB free; 5.40 GiB reserved in total by PyTorch)

In [10]:
!nvidia-smi

+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 50%   81C    P0    73W / 250W |     11MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:03:00.0 Off |                  N/A |
| 51%   82C    P0    70W / 250W |    137